<a href="https://colab.research.google.com/github/seyed-mohammadreza-mousavi/Retinal-Vessel-Segmentation_A-Computer-Vision-Technique/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/seyed-mohammadreza-mousavi/Retinal-Vessel-Segmentation_A-Computer-Vision-Technique.git
%cd Retinal-Vessel-Segmentation_A-Computer-Vision-Technique/ordered/

In [2]:
%run path.py
%run var.py
%run functions.py

In [3]:
print("number of training images:",len(train_image_path_list))
print("number of valid/test images:",len(val_image_path_list))
print("number of testing images:",len(test_image_path_list))

number of training images: 20
number of valid/test images: 20
number of testing images: 20


In [4]:
# generate train patch images
for i in tqdm(range(len(train_image_path_list)),desc="Generate the training patches: "):
  image2patch_train(train_image_path_list[i],patch_num,patch_size,training=True,show=False)  # set show=True to visualize the sample process, which is much slower than show=False

for i in tqdm(range(len(val_image_path_list)),desc="Generate the val patches: "):
  image2patch_validation(val_image_path_list[i],patch_num,patch_size,training=False,show=False)

train_patch_img_path_list=sorted(glob(train_patch_dir+"*-*-img.jpg"))
train_patch_groundtruth_path_list=sorted(glob(train_patch_dir+"*-*-groundtruth.jpg"))
train_patch_img_path_list,train_patch_groundtruth_path_list=shuffle(train_patch_img_path_list,train_patch_groundtruth_path_list,random_state=0)

val_patch_img_path_list=sorted(glob(test_patch_dir+"*_*_val_img.jpg"))
val_patch_groundtruth_path_list=sorted(glob(test_patch_dir+"*_*_val_groundtruth.jpg"))

print(len(train_patch_img_path_list),len(train_patch_groundtruth_path_list))
print(train_patch_img_path_list[:2])
print(train_patch_groundtruth_path_list[:2])

print(val_patch_img_path_list[:2])
print(val_patch_groundtruth_path_list[:2])

Generate the val patches: 100%|██████████| 20/20 [00:26<00:00,  1.31s/it]


30000 30000
['DRIVE/training/patch/26-300-img.jpg', 'DRIVE/training/patch/28-1262-img.jpg']
['DRIVE/training/patch/26-300-groundtruth.jpg', 'DRIVE/training/patch/28-1262-groundtruth.jpg']
['DRIVE/test/patch/01_0_val_img.jpg', 'DRIVE/test/patch/01_1000_val_img.jpg']
['DRIVE/test/patch/01_0_val_groundtruth.jpg', 'DRIVE/test/patch/01_1000_val_groundtruth.jpg']


In [73]:
import tensorflow as tf
from keras.layers import Conv2D, AveragePooling2D, BatchNormalization, Activation, Concatenate

class LinearTransform(tf.keras.layers.Layer):
  def __init__(self, name="LinearTransform", **kwargs):
    super(LinearTransform, self).__init__(self,name=name, **kwargs)

    self.conv_r=Conv2D(1,kernel_size=3,strides=1,padding='same',use_bias=False)
    self.conv_g=Conv2D(1,kernel_size=3,strides=1,padding='same',use_bias=False)
    self.conv_b=Conv2D(1,kernel_size=3,strides=1,padding='same',use_bias=False)

    self.pool_rc=AveragePooling2D(pool_size=(patch_size,patch_size),strides=1)
    self.pool_gc=AveragePooling2D(pool_size=(patch_size,patch_size),strides=1)
    self.pool_bc=AveragePooling2D(pool_size=(patch_size,patch_size),strides=1)

    self.bn=BatchNormalization()
    self.sigmoid=Activation('sigmoid')
    self.softmax=Activation('softmax')

  def call(self, input,training=True):
    r,g,b=input[:,:,:,0:1],input[:,:,:,1:2],input[:,:,:,2:3]

    rs=self.conv_r(r)
    gs=self.conv_g(g)
    bs=self.conv_r(b)

    rc=tf.reshape(self.pool_rc(rs),[-1,1])
    gc=tf.reshape(self.pool_gc(gs),[-1,1])
    bc=tf.reshape(self.pool_bc(bs),[-1,1])

    merge=Concatenate(axis=-1)([rc,gc,bc])
    merge=tf.expand_dims(merge,axis=1)
    merge=tf.expand_dims(merge,axis=1)
    merge=self.softmax(merge)
    merge=tf.repeat(merge,repeats=patch_size,axis=2)
    merge=tf.repeat(merge,repeats=patch_size,axis=1)

    r=r*(1+self.sigmoid(rs))
    g=g*(1+self.sigmoid(gs))
    b=b*(1+self.sigmoid(bs))

    output=self.bn(merge[:,:,:,0:1]*r+merge[:,:,:,1:2]*g+merge[:,:,:,2:3]*b,training=training)
    return output

In [80]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, AveragePooling2D, BatchNormalization, Activation, Concatenate, LeakyReLU


class LinearTransform(tf.keras.layers.Layer):
    def __init__(self, name="LinearTransform"):
        super(LinearTransform, self).__init__(name=name)
        self.patch_size = patch_size

        self.conv_r = Conv2D(1, kernel_size=3, strides=1, padding='same', use_bias=False)
        self.conv_g = Conv2D(1, kernel_size=3, strides=1, padding='same', use_bias=False)
        self.conv_b = Conv2D(1, kernel_size=3, strides=1, padding='same', use_bias=False)

        self.pool_rc = AveragePooling2D(pool_size=(patch_size, patch_size), strides=1)
        self.pool_gc = AveragePooling2D(pool_size=(patch_size, patch_size), strides=1)
        self.pool_bc = AveragePooling2D(pool_size=(patch_size, patch_size), strides=1)

        self.bn = BatchNormalization()
        self.sigmoid = Activation('sigmoid')
        self.softmax = Activation('softmax')

    def call(self, input, training=True):
        r, g, b = input[:, :, :, 0:1], input[:, :, :, 1:2], input[:, :, :, 2:3]

        rs = self.conv_r(r)
        gs = self.conv_g(g)
        bs = self.conv_r(b)

        rc = tf.reshape(self.pool_rc(rs), [-1, 1])
        gc = tf.reshape(self.pool_gc(gs), [-1, 1])
        bc = tf.reshape(self.pool_bc(bs), [-1, 1])

        merge = Concatenate(axis=-1)([rc, gc, bc])
        merge = tf.expand_dims(merge, axis=1)
        merge = tf.expand_dims(merge, axis=1)
        merge = self.softmax(merge)
        merge = tf.repeat(merge, repeats=self.patch_size, axis=2)
        merge = tf.repeat(merge, repeats=self.patch_size, axis=1)

        r = r * (1 + self.sigmoid(rs))
        g = g * (1 + self.sigmoid(gs))
        b = b * (1 + self.sigmoid(bs))

        output = self.bn(merge[:, :, :, 0:1] * r + merge[:, :, :, 1:2] * g + merge[:, :, :, 2:3], training=training)
        return output

    def get_config(self):
        config = super(LinearTransform, self).get_config()
        config.update({"patch_size": self.patch_size})
        return config

In [ ]:
LinearTransform()

In [68]:
input_tensor = tf.random.normal((1, 48, 48, 3))  # Example shape, adjust according to your needs

In [69]:
LinearTransform()(input_tensor).shape

TensorShape([1, 48, 48, 1])

In [ ]:
LinearTransform()(input_tensor)

In [81]:
class ResBlock(tf.keras.layers.Layer):
    def __init__(self, out_ch, residual_path=False, stride=1, **kwargs):
        super(ResBlock, self).__init__(**kwargs)
        self.residual_path = residual_path
        self.stride = stride

        self.conv1 = Conv2D(out_ch, kernel_size=3, strides=stride, padding='same', use_bias=False,
                            data_format="channels_last")
        self.bn1 = BatchNormalization()
        self.relu1 = LeakyReLU()

        self.conv2 = Conv2D(out_ch, kernel_size=3, strides=1, padding='same', use_bias=False,
                            data_format="channels_last")
        self.bn2 = BatchNormalization()

        if residual_path:
            self.conv_shortcut = Conv2D(out_ch, kernel_size=1, strides=stride, padding='same', use_bias=False)
            self.bn_shortcut = BatchNormalization()

        self.relu2 = LeakyReLU()

    def call(self, x, training=True):
        xs = self.relu1(self.bn1(self.conv1(x), training=training))
        xs = self.bn2(self.conv2(xs), training=training)

        if self.residual_path:
            x = self.bn_shortcut(self.conv_shortcut(x), training=training)

        xs = x + xs
        return self.relu2(xs)

    def get_config(self):
        config = super(ResBlock, self).get_config()
        config.update({"out_ch": self.conv1.filters,
                       "residual_path": self.residual_path,
                       "stride": self.stride})
        return config

In [ ]:
ResBlock(out_ch=64, residual_path=True, stride=1)(input_tensor)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Concatenate, MaxPool2D, UpSampling2D


class Unet(tf.keras.Model):
    def __init__(self):
        super(Unet, self).__init__()

        self.conv_init = LinearTransform()
        self.resinit = ResBlock(16, residual_path=True)
        self.up_sample = UpSampling2D(size=(2, 2), interpolation='bilinear')
        self.resup = ResBlock(32, residual_path=True)

        self.pool1 = MaxPool2D(pool_size=(2, 2))

        self.resblock_down1 = ResBlock(64, residual_path=True)
        self.resblock_down11 = ResBlock(64, residual_path=False)
        self.pool2 = MaxPool2D(pool_size=(2, 2))

        self.resblock_down2 = ResBlock(128, residual_path=True)
        self.resblock_down21 = ResBlock(128, residual_path=False)
        self.pool3 = MaxPool2D(pool_size=(2, 2))

        self.resblock_down3 = ResBlock(256, residual_path=True)
        self.resblock_down31 = ResBlock(256, residual_path=False)
        self.pool4 = MaxPool2D(pool_size=(2, 2))

        self.resblock = ResBlock(512, residual_path=True)

        self.unpool3 = UpSampling2D(size=(2, 2), interpolation='bilinear')
        self.resblock_up3 = ResBlock(256, residual_path=True)
        self.resblock_up31 = ResBlock(256, residual_path=False)

        self.unpool2 = UpSampling2D(size=(2, 2), interpolation='bilinear')
        self.resblock_up2 = ResBlock(128, residual_path=True)
        self.resblock_up21 = ResBlock(128, residual_path=False)

        self.unpool1 = UpSampling2D(size=(2, 2), interpolation='bilinear')
        self.resblock_up1 = ResBlock(64, residual_path=True)

        self.unpool_final = UpSampling2D(size=(2, 2), interpolation='bilinear')
        self.resblock2 = ResBlock(32, residual_path=True)

        self.pool_final = MaxPool2D(pool_size=(2, 2))
        self.resfinal = ResBlock(32)

        self.conv_final = Conv2D(1, kernel_size=1, strides=1, padding='same', use_bias=False)
        self.bn_final = BatchNormalization()
        self.act = Activation('sigmoid')

    def call(self, x, training=True):
        x_linear = self.conv_init(x, training=training)
        x = self.resinit(x_linear, training=training)
        x = self.up_sample(x)
        x = self.resup(x, training=training)

        stage1 = self.pool1(x)
        stage1 = self.resblock_down1(stage1, training=training)
        stage1 = self.resblock_down11(stage1, training=training)

        stage2 = self.pool2(stage1)
        stage2 = self.resblock_down2(stage2, training=training)
        stage2 = self.resblock_down21(stage2, training=training)

        stage3 = self.pool3(stage2)
        stage3 = self.resblock_down3(stage3, training=training)
        stage3 = self.resblock_down31(stage3, training=training)

        stage4 = self.pool4(stage3)
        stage4 = self.resblock(stage4, training=training)

        stage3 = Concatenate(axis=3)([stage3, self.unpool3(stage4)])
        stage3 = self.resblock_up3(stage3, training=training)
        stage3 = self.resblock_up31(stage3, training=training)

        stage2 = Concatenate(axis=3)([stage2, self.unpool2(stage3)])
        stage2 = self.resblock_up2(stage2, training=training)
        stage2 = self.resblock_up21(stage2, training=training)

        stage1 = Concatenate(axis=3)([stage1, self.unpool1(stage2)])
        stage1 = self.resblock_up1(stage1, training=training)

        x = Concatenate(axis=3)([x, self.unpool_final(stage1)])
        x = self.resblock2(x, training=training)

        x = self.pool_final(x)
        x = self.resfinal(x, training=training)

        seg_result = self.act(self.bn_final(self.conv_final(x), training=training))

        return x_linear, seg_result

In [ ]:
import tensorflow as tf

# Assuming you have defined the necessary layers and classes (LinearTransform, ResBlock, etc.)

# Create an instance of the Unet model
model = Unet()

# Define the input shape
input_shape = (256, 256, 3)  # Example input shape

# Create a placeholder input tensor
inputs = tf.keras.Input(shape=input_shape)

# Pass the input tensor through the Unet model
x_linear, seg_result = model(inputs)

# Create the Keras model
keras_model = tf.keras.Model(inputs=inputs, outputs=[x_linear, seg_result])

# Compile the model with an appropriate loss and optimizer
keras_model.compile(optimizer='adam', loss='binary_crossentropy')

# Train the model
keras_model.fit(x_train, y_train, epochs=10, batch_size=32)